# 🛠️ MVP - Predição de Manutenção Industrial com Machine Learning

Este notebook tem como objetivo construir um modelo de machine learning para prever falhas em equipamentos industriais com base em sensores operacionais. Utilizo o dataset **AI4I 2020 Predictive Maintenance**.

O modelo gerado será exportado e utilizado em uma aplicação web full stack desenvolvida com FastAPI.


### 1️⃣ Importação das bibliotecas

Nesta etapa, carregamos as bibliotecas necessárias para manipulação de dados, dividir amostras, treinar modelos de machine learning e exportar o resultado final.

In [ ]:

# 🔹 1. Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import joblib


### 2️⃣ Carregamento do dataset

O dataset **AI4I 2020** é carregado diretamente via URL. Ele contém dados simulados de sensores industriais (temperatura, torque, rotação etc.) e se uma falha de máquina ocorreu (`machine_failure`).

In [ ]:
# 🔹 2. Upload e carregamento do dataset AI4I 2020
from google.colab import files
import pandas as pd

uploaded = files.upload()  # selecione o arquivo ai4i2020.csv
df = pd.read_csv('ai4i2020.csv')

df.head()  # para verificar as primeiras linhas

### 3️⃣ Pré-processamento dos dados

Removemos colunas que não são úteis para o modelo (`udi`, `product_id`, `type`) e normalizamos os nomes das colunas para facilitar o trabalho com o Pandas e Scikit-learn.


In [ ]:
# 🔹 3. Pré-processamento básico
df.columns = [col.strip().lower().replace(" ", "_").replace("[", "").replace("]", "").replace("/", "") for col in df.columns]
df = df.drop(columns=['udi', 'product_id', 'type'])  # Remover colunas não numéricas
X = df.drop(columns=['machine_failure'])
y = df['machine_failure']

### 4️⃣ Divisão dos dados em treino e teste

Separamos os dados em 80% para treino e 20% para teste. Isso nos permite avaliar o desempenho real dos modelos em dados nunca vistos.


In [ ]:
# 🔹 4. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 5️⃣ Treinamento dos modelos

Utilizamos os seguintes algoritmos clássicos de classificação:

- K-Nearest Neighbors (KNN)
- Árvore de Decisão
- Naive Bayes
- Suporte a Vetores de Máquinas (SVM)

Todos são encapsulados em um pipeline com normalização via `StandardScaler`.


In [ ]:
# 🔹 5. Modelos
modelos = {
    "KNN": KNeighborsClassifier(),
    "Árvore": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC()
}

### 6️⃣ Avaliação dos resultados

Geramos relatórios de classificação (`classification_report`) para comparar os modelos usando métricas como **precisão**, **recall** e **F1-score**.

Essas métricas nos ajudam a escolher o modelo com melhor equilíbrio entre acertos e erros.


In [ ]:
# 🔹 6. Avaliação
resultados = {}
melhor_f1 = -1  # Inicializa com um valor baixo
melhor_modelo = None # Inicializa melhor_modelo como None

for nome, modelo in modelos.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        (nome, modelo)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True) # Obter o relatório como dicionário
    f1_score = report['1']['f1-score'] # Acessar o F1-score para a classe positiva ('1')

    print(f"\n🔍 {nome} - Relatório de Classificação")
    print(classification_report(y_test, y_pred))

    resultados[nome] = pipeline

    # Comparar e selecionar o melhor modelo com base no F1-score
    if f1_score > melhor_f1:
        melhor_f1 = f1_score
        melhor_modelo = pipeline # Armazenar o pipeline do melhor modelo

### 7️⃣ Exportação do melhor modelo

Selecionamos a Árvore de Decisão como o modelo final com base nos resultados das métricas. O modelo é salvo no formato `.pkl` para ser carregado pela aplicação FastAPI e utilizado em predições futuras.


In [ ]:
# 🔹 7. Exportar melhor modelo com pickle
import pickle
with open('modelo_manutencao.pkl', 'wb') as f:
    pickle.dump(melhor_modelo, f)
print('Modelo salvo como modelo_manutencao.pkl')

# Testar o carregamento
with open('modelo_manutencao.pkl', 'rb') as f:
    modelo_carregado = pickle.load(f)
print('Modelo carregado com sucesso:', modelo_carregado)


### 8️⃣ Exportação do modelo treinado (.pkl)
Após o pipeline de pré-processamento e treinamento, esta célula exporta o modelo para ser utilizado na API.
Este modelo é salvo com a versão 1.7.0 do scikit-learn para evitar incompatibilidades durante o uso.

In [ ]:
# 🔹 7. Exportar melhor modelo com pickle
import pickle
with open('modelo_manutencao.pkl', 'wb') as f:
    pickle.dump(melhor_modelo, f)
print('Modelo salvo como modelo_manutencao.pkl')

# Testar o carregamento
with open('modelo_manutencao.pkl', 'rb') as f:
    modelo_carregado = pickle.load(f)
print('Modelo carregado com sucesso:', modelo_carregado)


### 9️⃣ Download do modelo treinado
Nesta etapa realizamos o download do arquivo .pkl que contém o melhor modelo treinado para uso posterior.

In [ ]:
# 🔹 7. Exportar melhor modelo com pickle
import pickle
with open('modelo_manutencao.pkl', 'wb') as f:
    pickle.dump(melhor_modelo, f)
print('Modelo salvo como modelo_manutencao.pkl')

# Testar o carregamento
with open('modelo_manutencao.pkl', 'rb') as f:
    modelo_carregado = pickle.load(f)
print('Modelo carregado com sucesso:', modelo_carregado)


In [ ]:
from google.colab import files
files.download('modelo_manutencao.pkl')


## ✅ Conclusão

Este notebook permitiu a construção de um modelo de machine learning para manutenção preditiva industrial, utilizando o dataset AI4I 2020.

O modelo final foi treinado com a Árvore de Decisão, avaliado com métricas robustas, e exportado para ser utilizado na aplicação full stack que integra front-end (HTML/JS) com back-end em FastAPI.

Essa solução permite prever falhas com base nos sensores de temperatura, pressão, vibração e corrente elétrica.
